<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=0b0467e5d66777fc448e6025481a8787460554993ee4aae6f15a01467da21b5a
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-19 11:07:21
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.40 C
-------------------
Today PnL: 27.27 K (0.2%)
Current PnL: -27.53 L (-17.79%)
CY Booked + Current PnL: -13.24 L (-8.56%)
-------------------
Total profit:  1.42 L
Total loss:  -28.95 L
-------------------
Total Booked + Current PnL: 13.74 L (10.75%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.24%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 92.17 L (66.06%)
Deployed:  1.28 C
Current:  1.40 C
CAGR/XIRR %: 4.85%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,1530.00,-0.22,9.41,9.40,19.69,19720.0,18048.0,209790.0,-1.22,58.0,X-MC,2.54,78.0,0.92,1.51,16.85,XY25,NTT,AC
77,TTKPRESTIG,770.00,-0.68,-20.69,26.20,0.09,20942.0,-20847.0,79930.0,85.49,25.0,M-SC,2.30,253.0,-1.00,0.58,5.01,OX40N,NTT,DURABLES
50,MASFIN,398.61,1.87,-2.53,25.21,22.05,24077.0,-2475.0,95505.0,-15.99,53.0,H-SC,6.43,164.0,-0.10,0.69,38.82,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,0.36,-15.96,19.02,0.02,27901.0,-27867.0,146692.0,123.77,42.0,M-SC,13.11,234.0,-1.00,1.06,27.60,OX40N,NTT,PAINTS
43,ITC,452.00,0.12,-1.74,12.75,10.78,30107.0,-4188.0,236130.0,-41.56,45.0,X-LC,2.15,5.0,-0.14,1.70,3.40,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.0,4.88,-4.48,90.20,81.67,80824.0,-4205.0,89605.0,8066.67,71.0,L-SC,13.01,271.0,-0.05,0.65,73.52,XR,NTT,CERAMICS
72,TATAELXSI,9161.0,4.66,-24.38,74.60,32.03,74371.0,-32142.0,99693.0,-18.22,56.0,H-SC,9.65,148.0,-0.43,0.72,11.66,OX40N,NTT,IT
79,VAIBHAVGBL,521.0,2.72,-26.65,119.19,60.77,159783.0,-48718.0,134057.0,54.82,48.0,H-SC,4.87,159.0,-0.30,0.97,20.73,XR,NTT,JEWELLERY
10,BANDHANBNK,400.0,2.39,-28.42,168.80,92.42,336596.0,-79154.0,199405.0,40.09,49.0,H-SC,9.28,170.0,-0.24,1.44,15.24,XY24,NTT,BANKS
64,SHALBY,327.0,2.13,-25.00,66.80,25.10,90752.0,-45287.0,135856.0,898.41,37.0,M-SC,11.02,250.0,-0.50,0.98,8.87,XY24,NTT,HEALTHCARE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,TANLA,1943.92,-3.15,-38.24,255.74,119.69,427633.0,-103547.0,167214.0,-37.58,40.0,H-SC,7.50,156.0,-0.24,1.20,32.10,AR,ATH,IT
61,ROUTE,2227.21,-1.32,-45.33,209.23,69.05,152204.0,-60321.0,72745.0,-55.87,56.0,H-SC,21.95,140.0,-0.40,0.52,12.19,SR,ATH,IT
48,KPIGREEN,730.66,-1.11,-15.97,74.88,46.95,78838.0,-20011.0,105286.0,-22.21,42.0,H-SC,8.97,154.0,-0.25,0.76,31.90,MH,ATH,POWER
81,VBL,671.64,-1.01,-5.30,43.27,35.67,129418.0,-16748.0,299094.0,-16.55,48.0,X-LC,3.95,4.0,-0.13,2.15,7.82,X40N,ATH,FMCG
54,PGHH,17907.65,-0.98,-3.60,39.21,34.19,80704.0,-7696.0,205824.0,-32.24,49.0,X-MC,5.19,57.0,-0.10,1.48,2.40,X40,ATH,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,1.05,-0.83,109.66,107.91,160555.0,-1232.0,146412.0,-19.34,60.0,M-SC,12.28,216.0,-0.01,1.05,9.17,OX40N,ATH,CABLES


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,1.05,-0.83,109.66,107.91,160555.0,-1232.0,146412.0,-19.34,60.0,M-SC,12.28,216.0,-0.01,1.05,9.17,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.36,-15.96,19.02,0.02,27901.0,-27867.0,146692.0,123.77,42.0,M-SC,13.11,234.0,-1.00,1.06,27.60,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.68,-20.69,26.20,0.09,20942.0,-20847.0,79930.0,85.49,25.0,M-SC,2.30,253.0,-1.00,0.58,5.01,OX40N,NTT,DURABLES
66,SIS,528.00,0.05,-24.21,60.00,21.26,50490.0,-26882.0,84150.0,1984.97,50.0,H-SC,4.86,166.0,-0.53,0.61,13.70,OX40N,NTT,MISC
72,TATAELXSI,9161.00,4.66,-24.38,74.60,32.03,74371.0,-32142.0,99693.0,-18.22,56.0,H-SC,9.65,148.0,-0.43,0.72,11.66,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.4,8.65,58.55,72.27,107793.0,14657.0,184105.0,-6.53,70.0,M-LC,3.07,99.0,0.14,1.33,15.36,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.40,8.65,58.55,72.27,107793.0,14657.0,184105.0,-6.53,70.0,M-LC,3.07,99.0,0.14,1.33,15.36,XR,NTT,IT
25,FINCABLES,1641.55,1.05,-0.83,109.66,107.91,160555.0,-1232.0,146412.0,-19.34,60.0,M-SC,12.28,216.0,-0.01,1.05,9.17,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-0.62,-3.32,113.81,106.72,135713.0,-4091.0,119245.0,-52.15,40.0,H-SC,3.65,139.0,-0.03,0.86,19.29,AR,ATH,MISC
6,ASIANTILES,137.00,4.88,-4.48,90.20,81.67,80824.0,-4205.0,89605.0,8066.67,71.0,L-SC,13.01,271.0,-0.05,0.65,73.52,XR,NTT,CERAMICS
85,ZYDUSLIFE,1286.17,0.26,-3.40,40.40,35.63,81789.0,-7127.0,202447.0,-17.18,38.0,H-MC,2.65,119.0,-0.09,1.46,12.70,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PAGEIND,51605.07,-0.66,-11.97,45.53,28.11,80725.0,-24110.0,177300.0,-36.23,22.0,X-MC,13.82,55.0,-0.30,1.28,0.00,X40,ATH,APPARELS
8,AWL,485.00,-0.23,-24.46,103.18,53.50,235207.0,-73793.0,227958.0,-63.37,27.0,X-MC,4.24,58.0,-0.31,1.64,1.25,XY24,NTT,FMCG
9,BAJAJHFL,181.50,0.66,-18.05,89.71,55.47,178860.0,-43910.0,199376.0,-27.39,30.0,X-MC,14.35,64.0,-0.25,1.44,2.16,X40N,ATH,FINANCE
3,ACC,3906.00,0.07,-26.20,122.42,64.15,214982.0,-62341.0,175610.0,-56.75,31.0,X-SC,3.21,82.0,-0.29,1.26,0.07,XY24,BTT,CEMENT
23,DMART,5391.45,0.28,-9.43,43.09,29.60,97413.0,-23532.0,226068.0,-22.79,31.0,X-LC,6.74,19.0,-0.24,1.63,10.69,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,-0.54,-1.97,26.97,24.48,66591.0,-4950.0,246906.0,-81.32,44.0,X-SC,0.79,86.0,-0.07,1.78,20.72,X40N,NTT,MISC
12,BERGEPAINT,680.0,0.09,-4.36,26.66,21.14,57965.0,-9915.0,217424.0,-11.18,38.0,X-MC,1.33,74.0,-0.17,1.57,22.55,XY24,NTT,PAINTS
20,DABUR,735.0,-0.16,-3.86,49.47,43.70,117985.0,-9569.0,238499.0,-11.80,33.0,X-MC,2.02,72.0,-0.08,1.72,10.61,XY24,BTT,FMCG
43,ITC,452.0,0.12,-1.74,12.75,10.78,30107.0,-4188.0,236130.0,-41.56,45.0,X-LC,2.15,5.0,-0.14,1.70,3.40,X40,NTT,FMCG
78,UNITDSPR,1644.0,0.08,2.28,18.08,20.77,43297.0,5332.0,239476.0,-6.55,40.0,X-MC,2.22,62.0,0.12,1.72,9.35,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COLPAL,3726.84,-0.45,-21.00,79.12,41.51,164617.0,-55305.0,208060.0,-5.86,34.0,X-MC,7.68,61.0,-0.34,1.50,0.00,XY25,ATH,FMCG
53,PAGEIND,51605.07,-0.66,-11.97,45.53,28.11,80725.0,-24110.0,177300.0,-36.23,22.0,X-MC,13.82,55.0,-0.30,1.28,0.00,X40,ATH,APPARELS
3,ACC,3906.00,0.07,-26.20,122.42,64.15,214982.0,-62341.0,175610.0,-56.75,31.0,X-SC,3.21,82.0,-0.29,1.26,0.07,XY24,BTT,CEMENT
11,BATAINDIA,2096.00,0.35,-39.15,122.18,35.20,95669.0,-50368.0,78302.0,2.67,32.0,X-SC,16.62,92.0,-0.53,0.56,0.44,X40,NTT,FOOTWEAR
31,HAVELLS,2069.16,-0.12,-10.40,47.85,32.48,140629.0,-34096.0,293895.0,-14.91,40.0,X-MC,6.40,67.0,-0.24,2.12,0.69,X40,ATH,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,1.14,-29.91,103.80,42.84,47294.0,-19443.0,45563.0,-54.27,43.0,X-SC,34.78,83.0,-0.41,0.33,3.08,XY24,NTT,MISC
59,RELAXO,1176.00,-0.35,-48.03,193.05,52.31,145645.0,-69716.0,75444.0,-44.63,41.0,X-SC,7.30,91.0,-0.48,0.54,0.99,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.35,-39.15,122.18,35.20,95669.0,-50368.0,78302.0,2.67,32.0,X-SC,16.62,92.0,-0.53,0.56,0.44,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,0.33,-3.44,28.50,24.08,34935.0,-4372.0,122578.0,-11.26,38.0,X-SC,5.59,89.0,-0.13,0.88,15.50,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,0.79,-16.77,72.20,43.32,97874.0,-27312.0,135560.0,-28.43,38.0,X-SC,5.06,90.0,-0.28,0.98,4.09,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.89,-8.99,32.63,20.71,112326.0,-33997.0,344240.0,-21.68,71.0,X-LC,3.86,1.0,-0.30,2.48,15.02,X40,ATH,IT
41,INFY,1972.00,1.30,12.19,19.66,34.24,69011.0,38136.0,351024.0,-12.78,70.0,X-LC,4.92,2.0,0.55,2.53,21.48,X40,NTT,IT
75,TMPV,600.00,1.40,-28.13,71.11,22.98,115448.0,-63547.0,162351.0,-25.52,39.0,X-LC,6.84,3.0,-0.55,1.17,2.11,XY24,NTT,AUTO
81,VBL,671.64,-1.01,-5.30,43.27,35.67,129418.0,-16748.0,299094.0,-16.55,48.0,X-LC,3.95,4.0,-0.13,2.15,7.82,X40N,ATH,FMCG
43,ITC,452.00,0.12,-1.74,12.75,10.78,30107.0,-4188.0,236130.0,-41.56,45.0,X-LC,2.15,5.0,-0.14,1.70,3.40,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,4.88,-4.48,90.20,81.67,80824.0,-4205.0,89605.0,8066.67,71.0,L-SC,13.01,271.0,-0.05,0.65,73.52,XR,NTT,CERAMICS
50,MASFIN,398.61,1.87,-2.53,25.21,22.05,24077.0,-2475.0,95505.0,-15.99,53.0,H-SC,6.43,164.0,-0.10,0.69,38.82,XR,ATH,FINANCE
13,BSOFT,831.70,-0.08,-21.50,92.08,50.79,101670.0,-30238.0,110415.0,0.89,65.0,H-SC,6.88,151.0,-0.30,0.80,28.56,XR,ATH,IT
39,INDIGOPNTS,1408.00,0.36,-15.96,19.02,0.02,27901.0,-27867.0,146692.0,123.77,42.0,M-SC,13.11,234.0,-1.00,1.06,27.60,OX40N,NTT,PAINTS
84,WIPRO,420.00,0.40,8.65,58.55,72.27,107793.0,14657.0,184105.0,-6.53,70.0,M-LC,3.07,99.0,0.14,1.33,15.36,XR,NTT,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,-0.08,-21.50,92.08,50.79,101670.0,-30238.0,110415.0,0.89,65.0,H-SC,6.88,151.0,-0.30,0.80,28.56,XR,ATH,IT
84,WIPRO,420.00,0.40,8.65,58.55,72.27,107793.0,14657.0,184105.0,-6.53,70.0,M-LC,3.07,99.0,0.14,1.33,15.36,XR,NTT,IT
6,ASIANTILES,137.00,4.88,-4.48,90.20,81.67,80824.0,-4205.0,89605.0,8066.67,71.0,L-SC,13.01,271.0,-0.05,0.65,73.52,XR,NTT,CERAMICS
41,INFY,1972.00,1.30,12.19,19.66,34.24,69011.0,38136.0,351024.0,-12.78,70.0,X-LC,4.92,2.0,0.55,2.53,21.48,X40,NTT,IT
61,ROUTE,2227.21,-1.32,-45.33,209.23,69.05,152204.0,-60321.0,72745.0,-55.87,56.0,H-SC,21.95,140.0,-0.40,0.52,12.19,SR,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.41
1,25,44.87
2,50,76.61


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.88
MC    30.85
LC    24.30
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.65
X40      23.17
X40N     12.95
XR        9.87
XY25      9.10
AR        8.96
OX40N     7.56
SR        1.01
MH        0.76
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.37
H-SC    23.72
X-LC    20.84
M-SC    11.62
X-SC     8.05
H-MC     4.80
L-SC     1.49
M-MC     1.34
M-LC     1.33
H-LC     1.11
L-LC     1.02
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.27,-8.84,43.84
IT,13.22,-14.49,72.06
FINANCE,9.48,-21.13,73.81
MISC,7.27,-28.98,81.73
ELECTRICAL,5.98,-13.00,54.39
PAINTS,5.82,-14.30,31.28
INSURANCE,4.80,-1.59,36.35
PHARMA,4.14,-2.31,34.52
AUTO,2.83,-33.33,79.55


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3223787.0,21
AR,1325927.0,10
XR,1296107.0,13
X40,1005506.0,14
X40N,886443.0,9
OX40N,741965.0,10
XY25,378390.0,6
SR,279980.0,2
MH,78838.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3686849.0,25
M-SC,1447370.0,15
X-MC,1442726.0,16
X-LC,832935.0,11
X-SC,800956.0,8
H-MC,412052.0,3
L-SC,255522.0,3
M-LC,107793.0,1
H-LC,78817.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1296702.0      6
           AR         904140.0      5
M-SC       XY24       828256.0      6
H-SC       XR         787368.0      7
X-MC       X40        497730.0      7
           XY24       411157.0      3
           X40N       349502.0      4
H-SC       OX40N      339821.0      4
M-SC       OX40N      315939.0      5
X-LC       X40        314141.0      5
X-SC       X40N       310110.0      3
           XY24       297211.0      3
H-SC       SR         279980.0      2
X-LC       X40N       226831.0      2
H-MC       AR         216205.0      2
           XY24       195847.0      1
X-LC       XY24       194614.0      2
X-SC       X40        193635.0      2
X-MC       XY25       184337.0      2
M-SC       XR         176410.0      2
L-SC       XR         169317.0      2
M-SC       AR         126765.0      2
M-LC       XR         107793.0      1
X-LC       XY25        97349.0      2
L-SC       OX40N       86205.0      1
H-SC       MH          78838.0      1
H-LC       AR          78817.0      1
M-MC       XY25        59281.0      1
L-MC       XR          55219.0      1
L-LC       XY25        37423.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
